In [1]:
import pandas as pd
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, HourLocator
from IPython.display import HTML

d = "Data//"
flis = ["LT1", "LT2", "LT3", "LT4", "LT5", "LT6", "LT7", "LT8"]
sheet = ["Leq","L90(LN4)"]
llist = ['None','Load','Unit 1', 'Unit 2', 'Unit 3', 'Unit 4']
load = pd.read_excel(d+"Load.xlsx")
load['Time'] = pd.to_datetime(load['Time'].astype(str))
weather = pd.read_excel(d+"Weather.xlsx")
weather['Time'] = pd.to_datetime(weather['Time'].astype(str))
wdir = ["N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW","---"]
lt = [[[] for i in range(2)] for j in range(8)]
dca, dcb = '07:00', '04:00'
lcrit = 1

for i, j in enumerate(flis):
    for k,l in enumerate(sheet):
        lt[i][k] = pd.read_excel(d+flis[i]+".xlsx", sheet_name=l)
        lt[i][k]['Start Time'] = pd.to_datetime(lt[i][k]['Start Time'].astype(str))
        lt[i][k] = pd.merge_asof(lt[i][k],load,left_on="Start Time", right_on="Time")
        lt[i][k] = lt[i][k].drop(labels=['Time'],axis=1)
        lt[i][k] = pd.merge_asof(lt[i][k],weather,left_on="Start Time", right_on="Time")
        lt[i][k] = lt[i][k].drop(labels=['Time'],axis=1)
        lt[i][k] = lt[i][k].set_index('Start Time')
        lt[i][k] = lt[i][k].tail(lt[i][k].shape[0] - 24) # removing first 6 hours from tail
        lt[i][k] = lt[i][k].head(lt[i][k].shape[0] - 24) # removing first 6 hours from head

%matplotlib widget
def table(n, lw, d, c, l, w):
    n = int(n-1)
    if lw == 'Leq':
        lwv = 0
        ax1.set_xlabel('L$_{EQ}$ dBA')
    else:
        lwv = 1
        ax1.set_xlabel('L$_{90}$ dBA')
    if d == 'Night':
        st, et  = '23:00', '07:00'
    else:
        st, et = '07:00', '23:00'
    ltall = lt[n][lwv].between_time(st, et) #filtering results for day and night
    ltall = ltall[ltall["Rain"]<0.2]
    ltall = ltall[ltall["Speed"]<=5]
    ltall = ltall[ltall["Dir"].isin(w)]
    ltdcall = ltall.between_time(dca, dcb) #filtering for dawn chorus
    if l == 'None':
        if c:
            lton_av,lton_min,lton_max,ltoff_av,ltoff_min,ltoff_max = 0,0,0,0,0,0
            ltall_av,ltall_min,ltall_max = ltdcall['Main'].mean(),ltdcall['Main'].min(),ltdcall['Main'].max()
        else:
            lton_av,lton_min,lton_max,ltoff_av,ltoff_min,ltoff_max = 0,0,0,0,0,0
            ltall_av,ltall_min,ltall_max = ltall['Main'].mean(),ltall['Main'].min(),ltall['Main'].max()
    else:
        lton, ltoff = ltall[ltall[l]>=lcrit], ltall[ltall[l]<lcrit] #filtering results with and without drax power
        ltdcon,ltdcoff = ltdcall[ltdcall[l]>=lcrit],ltdcall[ltdcall[l]<lcrit] #filtering dawn chorus results with and without drax power
        if c:
            lton_av,lton_min,lton_max = ltdcon['Main'].mean(),ltdcon['Main'].min(),ltdcon['Main'].max()
            ltoff_av,ltoff_min,ltoff_max = ltdcoff['Main'].mean(),ltdcoff['Main'].min(),ltdcoff['Main'].max()
            ltall_av,ltall_min,ltall_max = ltdcall['Main'].mean(),ltdcall['Main'].min(),ltdcall['Main'].max()
        else:
            lton_av,lton_min,lton_max = lton['Main'].mean(),lton['Main'].min(),lton['Main'].max()
            ltoff_av,ltoff_min,ltoff_max = ltoff['Main'].mean(),ltoff['Main'].min(),ltoff['Main'].max()
            ltall_av,ltall_min,ltall_max = ltall['Main'].mean(),ltall['Main'].min(),ltall['Main'].max()
    id1 = ['All data', 'Plant On','Plant Off']
    av1 = [ltall_av,lton_av, ltoff_av]
    min1 = [ltall_min, lton_min, ltoff_min]
    max1 = [ltall_max, lton_max, ltoff_max]
    df = {'ID':id1,'Arithmetic Mean (dBA)':av1,'Min (dBA)':min1,'Max (dBA)':max1}
    table1 = pd.DataFrame(df).round(1)
    display(HTML(table1.to_html(index=False,notebook=True)))

fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(10,4))
def plothist(n, lw, d, c, l, w):
    ax1.clear(), ax2.clear()
    n = int(n-1)
    bins = range(15,80)
    if lw == 'Leq':
        lwv = 0
        ax1.set_xlabel('L$_{EQ}$ dBA')
        ax2.set_xlabel('L$_{EQ}$ dBA')
    else:
        lwv = 1
        ax1.set_xlabel('L$_{90}$ dBA')
        ax2.set_xlabel('L$_{90}$ dBA')
    if d == 'Night':
        st, et  = '23:00', '07:00'
    else:
        st, et = '07:00', '23:00'
    ltall = lt[n][lwv].between_time(st, et) #filtering results for day and night
    ltall = ltall[ltall["Rain"]<0.2]
    ltall = ltall[ltall["Speed"]<=5]
    ltall = ltall[ltall["Dir"].isin(w)]
    ltdcall = ltall.between_time(dca, dcb) #filtering for dawn chorus
    if l == 'None':
        if c:
            ax1.hist(ltdcall['Main'],bins=bins,alpha=0.7)
            ax2.hist(ltdcall['Main'],bins=bins,alpha=0.7,density=True, histtype='step', cumulative=-1)
        else:
            ax1.hist(ltall['Main'],bins=bins,alpha=0.7)
            ax2.hist(ltall['Main'],bins=bins,alpha=0.7,density=True, histtype='step', cumulative=-1)
    else:
        lton, ltoff = ltall[ltall[l]>=lcrit], ltall[ltall[l]<lcrit] #filtering results with and without drax power
        ltdcon,ltdcoff = ltdcall[ltdcall[l]>=lcrit],ltdcall[ltdcall[l]<lcrit] #filtering dawn chorus results with and without drax power
        if c:
            ax1.hist(ltdcoff['Main'],bins=bins,alpha=0.7)
            ax1.hist(ltdcon['Main'],bins=bins,alpha=0.7)
            ax2.hist(ltdcoff['Main'],bins=bins,alpha=0.7,density=True, histtype='step', cumulative=-1)
            ax2.hist(ltdcon['Main'],bins=bins,alpha=0.7,density=True, histtype='step', cumulative=-1)
        else:
            ax1.hist(ltoff['Main'],bins=bins,alpha=0.7)
            ax1.hist(lton['Main'],bins=bins,alpha=0.7)
            ax2.hist(ltoff['Main'],bins=bins,alpha=0.7,density=True, histtype='step', cumulative=-1)
            ax2.hist(lton['Main'],bins=bins,alpha=0.7,density=True, histtype='step', cumulative=-1)
        ax1.legend(['Load <'+str(lcrit), 'Load >='+str(lcrit)])
    ax1.set_ylabel('Samples over survey period')
    ax2.set_ylabel('Probability (exceedance)')
    fig.show()

box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='stretch')
box_layout2 = widgets.Layout(display='inline-flex',flex_flow='row',align_items='stretch',margin='0px 0px 10px 45px')
ss = widgets.Select(options=llist,description='Load value: ',disabled=False)
aw = widgets.Select(options=list(range(1,9)),description='LT: ',disabled=False)
lw = widgets.ToggleButtons(options=['Leq','L90'],description='Stat. level: ',disabled=False, layout=dict(width='250px'))
bw = widgets.ToggleButtons(options=['Day','Night'],description='Period: ',disabled=False, layout=dict(width='250px'))
cw = widgets.Checkbox(value=False,description='Filter dawn Chorus',disabled=False)
wd = widgets.SelectMultiple(options=wdir,value=[wdir[0]],description='wind dir', disabled=False)
out = widgets.interactive_output(plothist, {'n': aw, 'lw': lw, 'd': bw, 'c': cw, 'l':ss,'w':wd})
outt = widgets.interactive_output(table, {'n': aw, 'lw': lw, 'd': bw, 'c': cw, 'l':ss,'w':wd})
ui = widgets.VBox([widgets.HBox([lw, bw, cw],layout=box_layout2),widgets.HBox([aw, ss, wd])],layout=box_layout)
ui2 = widgets.HBox([outt],layout=widgets.Layout(margin='0px 0px 0px 90px'))
display(ui2,ui,out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()